#Environment Setting

In [0]:
#List of dependencies
import os
import shutil
import copy
import pandas as pd
import paramiko
import stat
import datetime

from fuzzywuzzy import fuzz
from fuzzywuzzy import process



#Extract 3 days' data

In [0]:
import paramiko
import stat
import datetime
import os
from stat import S_ISDIR, S_ISREG 
 
host = "xxxx"
username = "xxxx"
password = "xxxx"
port = 22

transport = paramiko.Transport((host, port))

remote_path = "xxxx"
local_path = "xxxx"

transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)


for i in os.listdir(PATH + local_path + "xxxx"):
    if i.split('.')[1] == 'csv':
        os.remove(PATH + local_path + "xxxx" + i)

def sftp_get_recursive(path, dest, sftp):
    subfolder_list = sftp.listdir_attr(path)
    dest = str(dest)
    
    if not os.path.isdir(PATH+dest):
        print("local folder does not exist")
        os.makedirs(PATH+dest, exist_ok=True)
      
    for subfolder in subfolder_list:
        print("subfolder: "+subfolder.filename)
        item_list = sftp.listdir_attr(path+subfolder.filename)


        #################create SFTP time for the file
        df = pd.DataFrame(columns=['FileName','SFTP_Time'])
        for item, i in zip(item_list,range(len(item_list))):
            if stat.S_ISDIR(item.st_mode):
                item_time = datetime.datetime.fromtimestamp(item.st_mtime)
            else:
                item_time = datetime.datetime.fromtimestamp(item.st_mtime)

            file_name = item.filename
            sftp_time = item_time
            df.loc[i] = [file_name,sftp_time]
        df.to_csv(PATH+dest + "/" + "xxxx" + subfolder.filename+'_time.csv', index=False)

        for item in item_list:
            if item.filename == 'images':
                if (datetime.datetime.now().timetuple().tm_yday - datetime.datetime.fromtimestamp(item.st_mtime).timetuple().tm_yday) <=3:
                    if stat.S_ISDIR(item.st_mode):
                        if os.path.exists(PATH+dest + "/" + "images/" + subfolder.filename):
                            for root, dirs, files in os.walk(PATH+dest + "/" + "images/" + subfolder.filename, topdown=False):
                                for file in files:
                                    file_path = os.path.join(root, file)
                                    os.remove(file_path)
                                for dir in dirs:
                                    dir_path = os.path.join(root, dir)
                                    os.rmdir(dir_path)
                            # Remove the empty folder
                            os.rmdir(PATH+dest + "/" + "images/" + subfolder.filename)
                            os.mkdir(PATH+dest + "/" + "images/" + subfolder.filename)

                        else:
                            os.mkdir(PATH+dest + "/" + "images/" + subfolder.filename)
                        for i in sftp.listdir_attr(path+subfolder.filename+"/"+item.filename):
                            sftp.get(path + subfolder.filename+ "/" + item.filename + "/" + i.filename, PATH+dest + "/" + "images/" + subfolder.filename + "/" + i.filename)


            if os.path.splitext(item.filename)[1]== ".csv":
                if (datetime.datetime.now().timetuple().tm_yday - datetime.datetime.fromtimestamp(item.st_mtime).timetuple().tm_yday) <=3:
                    sftp.get(path + subfolder.filename+ "/" + item.filename, PATH+dest + "/" + "camcom-data/" + item.filename)

    

sftp_get_recursive(remote_path, local_path, sftp)

sftp.close()
transport.close()

#Split data to failed/completed/review_completed

In [0]:
def clear_folder_except_file(folder_path, file_to_keep):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        if filename != file_to_keep and os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Removed file: {filename}")

# Usage example
folder_path = PATH + local_path + "/camcom-completed"
file_to_keep = 'temp.txt'
clear_folder_except_file(folder_path, file_to_keep)

folder_path = PATH + local_path + "/camcom-review failed"
file_to_keep = 'temp.txt'
clear_folder_except_file(folder_path, file_to_keep)



In [0]:
def extract_review_completed(folder_path, output_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            base_name = os.path.splitext(filename)[0]
            review_completed_file = base_name + '_review_completed.csv'
            review_completed_path = os.path.join(folder_path, review_completed_file)

            if os.path.isfile(review_completed_path):
                source_path = os.path.join(folder_path, filename)
                destination_path = os.path.join(output_path, filename)
                shutil.copy(source_path, output_path)

                source_completed_path = os.path.join(folder_path, review_completed_file)
                destination_completed_path = os.path.join(output_path, review_completed_file)
                shutil.copy(source_completed_path, output_path)

folder_path = PATH + local_path + "/camcom-data"
output_path = PATH + local_path + "/camcon-review completed"
extract_review_completed(folder_path, output_path)

def extract_completed(folder1_path, folder2_path):

    for filename in os.listdir(folder1_path):
        if filename.endswith('.csv') and '_review_completed' not in filename:
            base_file_path = os.path.join(folder1_path, filename)
            review_completed_file = filename.replace('.csv', '_review_completed.csv')
            review_completed_path = os.path.join(folder1_path, review_completed_file)

            if not os.path.isfile(review_completed_path):
                with open(base_file_path, 'r') as base_file:
                    content = base_file.read()
                    if 'failed' not in content and 'completed' in content:
                        destination_path = os.path.join(folder2_path, filename)
                        shutil.copy(base_file_path, destination_path)

folder1_path = PATH + local_path + "/camcom-data"
folder2_path = PATH + local_path + "/camcom-completed"
extract_completed(folder1_path, folder2_path)

def extract_failed(folder1_path, folder2_path, folder3_path, folder4_path):
    folder2_files = set(os.listdir(folder2_path))
    folder3_files = set(os.listdir(folder3_path))

    for filename in os.listdir(folder1_path):
        if filename not in folder2_files and filename not in folder3_files:
            source_path = os.path.join(folder1_path, filename)
            destination_path = os.path.join(folder4_path, filename)
            shutil.copy(source_path, destination_path)

folder1_path = PATH + local_path + "/camcom-data"
folder2_path = PATH + local_path + "/camcon-review completed"
folder3_path = PATH + local_path + "/camcom-completed"
folder4_path = PATH + local_path + "/camcom-review failed" 
extract_failed(folder1_path, folder2_path, folder3_path, folder4_path)


def move_files(file_list, folder1_path, folder2_path):
    for filename in file_list:
        source_path = os.path.join(folder1_path, filename)
        destination_path = os.path.join(folder2_path, filename)
        shutil.move(source_path, destination_path)
        print(f"Moved file: {filename} to {folder2_path}")

# Usage example
file_list = ["xxxx"]
folder1_path = PATH + local_path + "/camcom-review failed" 
folder2_path = PATH + local_path + "/camcom-completed"
move_files(file_list, folder1_path, folder2_path)






In [0]:
def count_files_in_folder(folder_path):
    file_count = 0
    for _, _, files in os.walk(folder_path):
        file_count += len(files)
    return file_count


folder_path = PATH + local_path + "/camcon-review completed"
total_files_1 = (count_files_in_folder(folder_path)-1)/2
print(f"Total number of files in {folder_path}: {total_files_1}")

folder_path = PATH + local_path + "/camcom-completed"
total_files_2 = count_files_in_folder(folder_path) -1
print(f"Total number of files in {folder_path}: {total_files_2}")

folder_path = PATH + local_path + "/camcom-review failed"
total_files_3 = count_files_in_folder(folder_path)-2 
print(f"Total number of files in {folder_path}: {total_files_3}")

folder_path = PATH + local_path + "/camcom-data"
total_files = count_files_in_folder(folder_path)-2-total_files_1
print(f"Total number of files in {folder_path}: {total_files}")

print(total_files_1+total_files_2+total_files_3)

